# Modeling

### Import Final Data

In [1]:
# Import packages
import pandas as pd

# Assuming you have a DataFrame `df` with features `X` and a target column `y`
X_train = pd.read_csv("../Final_Data/X_train.csv")
y_train = pd.read_csv("../Final_Data/y_train.csv")
X_test = pd.read_csv("../Final_Data/X_test.csv")
y_test = pd.read_csv("../Final_Data/y_test.csv")

# Now you have your train and test datasets ready
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(1534, 49) (2636, 49) (1534, 1) (2636, 1)


### Feature Scaling

In [2]:
from sklearn.preprocessing import StandardScaler

categorical_columns = [
    "country_United_States",
    "country_Saudi_Arabia",
    "country_India",
    "country_Britain",
    "country_Italy",
]

# Separate continuous and categorical columns
X_categorical = X_train[categorical_columns]
X_continuous = X_train.drop(columns=categorical_columns)

# Apply scaling only to continuous columns
scaler = StandardScaler()
X_continuous_scaled = pd.DataFrame(scaler.fit_transform(X_continuous))

# Concatenate the scaled continuous columns with the unchanged categorical columns
X_train_scaled = pd.concat(
    [X_continuous_scaled, X_categorical.reset_index(drop=True)], axis=1
)

# Apply the same transformation to the test set
X_categorical_test = X_test[categorical_columns]
X_continuous_test = X_test.drop(columns=categorical_columns)

X_continuous_test_scaled = pd.DataFrame(scaler.transform(X_continuous_test))
X_test_scaled = pd.concat(
    [X_continuous_test_scaled, X_categorical_test.reset_index(drop=True)], axis=1
)

# Ensure that all column names are strings
X_train_scaled.columns = X_train_scaled.columns.astype(str)
X_test_scaled.columns = X_test_scaled.columns.astype(str)

ModuleNotFoundError: No module named 'sklearn'

### Different Models

#### Neural Network

In [ ]:
# Import necessary libraries
import numpy as np
from sklearn.neural_network import MLPClassifier
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    roc_auc_score,
    auc,
    precision_recall_curve,
)
from sklearn.model_selection import GridSearchCV

In [ ]:
# Define parameter grid for GridSearchCV
param_grid = {
    "hidden_layer_sizes": [(50,), (100,), (150,), (50, 50), (100, 50)],
    "activation": ["relu", "tanh", "logistic"],
    "solver": ["adam", "sgd"],
    "alpha": [0.0001, 0.001, 0.01],
    "learning_rate": ["constant", "adaptive"],
}

# Initialize MLPClassifier and GridSearchCV
mlp = MLPClassifier(max_iter=500, random_state=42)
grid_search = GridSearchCV(mlp, param_grid, cv=5, scoring="recall", n_jobs=-1)
grid_search.fit(X_train_scaled, y_train)

# Best model from grid search
best_mlp = grid_search.best_estimator_

# Predict using best model
y_pred = best_mlp.predict(X_test_scaled)
y_pred_prob = best_mlp.predict_proba(X_test_scaled)[:, 1]

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_prob)

# Calculate PR AUC
precision_vals, recall_vals, _ = precision_recall_curve(y_test, y_pred_prob)
pr_auc = auc(recall_vals, precision_vals)

# Create a DataFrame for metrics
metrics_df = pd.DataFrame(
    {
        "Metric": ["Accuracy", "Precision", "Recall", "F1 Score", "ROC AUC", "PR AUC"],
        "Value": [accuracy, precision, recall, f1, roc_auc, pr_auc],
    }
)

# Display the best model and metrics
print("Best Model Parameters:", grid_search.best_params_)
metrics_df

In [ ]:
# Plot the confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
conf_matrix_reversed = np.flip(conf_matrix, axis=(0, 1))

plt.figure(figsize=(8, 6))
sns.heatmap(
    conf_matrix_reversed,
    annot=True,
    fmt="d",
    cmap="Blues",
    cbar=False,
    xticklabels=["Predicted Positive", "Predicted Negative"],
    yticklabels=["Actual Positive", "Actual Negative"],
)
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
# Plot Precision-Recall (PR) curve
plt.figure()
plt.plot(
    recall_vals,
    precision_vals,
    color="orange",
    lw=2,
    label=f"PR curve (AUC = {pr_auc:.2f})",
)
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall (PR) Curve")
plt.legend(loc="lower left")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.grid(True)
plt.show()

#### K Nearest Neighbor

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# Define parameter grid for GridSearchCV
param_grid = {
    "n_neighbors": [3, 5, 7, 9, 11, 13],
    "weights": ["uniform", "distance"],
    "metric": ["euclidean", "manhattan", "minkowski"],
}

# Initialize KNeighborsClassifier and GridSearchCV
knn = KNeighborsClassifier()
grid_search = GridSearchCV(knn, param_grid, cv=5, scoring="recall", n_jobs=-1)
grid_search.fit(X_train_scaled, y_train)

# Best model from grid search
best_knn = grid_search.best_estimator_

# Predict using best model
y_pred = best_knn.predict(X_test_scaled)
y_pred_prob = best_knn.predict_proba(X_test_scaled)[:, 1]

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_prob)

# Calculate PR AUC
precision_vals, recall_vals, _ = precision_recall_curve(y_test, y_pred_prob)
pr_auc = auc(recall_vals, precision_vals)

# Create a DataFrame for metrics
metrics_df = pd.DataFrame(
    {
        "Metric": ["Accuracy", "Precision", "Recall", "F1 Score", "ROC AUC", "PR AUC"],
        "Value": [accuracy, precision, recall, f1, roc_auc, pr_auc],
    }
)

# Display the best model and metrics
print("Best Model Parameters:", grid_search.best_params_)
metrics_df

In [ ]:
# Plot the confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
conf_matrix_reversed = np.flip(conf_matrix, axis=(0, 1))

plt.figure(figsize=(8, 6))
sns.heatmap(
    conf_matrix_reversed,
    annot=True,
    fmt="d",
    cmap="Blues",
    cbar=False,
    xticklabels=["Predicted Positive", "Predicted Negative"],
    yticklabels=["Actual Positive", "Actual Negative"],
)
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
# Plot Precision-Recall (PR) curve
plt.figure()
plt.plot(
    recall_vals,
    precision_vals,
    color="orange",
    lw=2,
    label=f"PR curve (AUC = {pr_auc:.2f})",
)
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall (PR) Curve")
plt.legend(loc="lower left")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.grid(True)
plt.show()

#### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Define parameter grid for GridSearchCV
param_grid = {
    "n_estimators": [50, 100, 200],
    "max_depth": [None, 10, 20, 30],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "max_features": ["sqrt", "log2"],
}

# Initialize RandomForestClassifier and GridSearchCV
rf = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(rf, param_grid, cv=5, scoring="recall", n_jobs=-1)
grid_search.fit(X_train_scaled, y_train)

# Best model from grid search
best_rf = grid_search.best_estimator_

# Predict using best model
y_pred = best_rf.predict(X_test_scaled)
y_pred_prob = best_rf.predict_proba(X_test_scaled)[:, 1]

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_prob)

# Calculate PR AUC
precision_vals, recall_vals, _ = precision_recall_curve(y_test, y_pred_prob)
pr_auc = auc(recall_vals, precision_vals)

# Create a DataFrame for metrics
metrics_df = pd.DataFrame(
    {
        "Metric": ["Accuracy", "Precision", "Recall", "F1 Score", "ROC AUC", "PR AUC"],
        "Value": [accuracy, precision, recall, f1, roc_auc, pr_auc],
    }
)

# Display the best model and metrics
print("Best Model Parameters:", grid_search.best_params_)
metrics_df

In [ ]:
# Plot the confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
conf_matrix_reversed = np.flip(conf_matrix, axis=(0, 1))

plt.figure(figsize=(8, 6))
sns.heatmap(
    conf_matrix_reversed,
    annot=True,
    fmt="d",
    cmap="Blues",
    cbar=False,
    xticklabels=["Predicted Positive", "Predicted Negative"],
    yticklabels=["Actual Positive", "Actual Negative"],
)
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
# Plot Precision-Recall (PR) curve
plt.figure()
plt.plot(
    recall_vals,
    precision_vals,
    color="orange",
    lw=2,
    label=f"PR curve (AUC = {pr_auc:.2f})",
)
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall (PR) Curve")
plt.legend(loc="lower left")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.grid(True)
plt.show()

#### Logisitic Regression

In [ ]:
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

# Identify numeric and categorical columns
numeric_cols = X_train.select_dtypes(include=["int64", "float64"]).columns
categorical_cols = X_train.select_dtypes(include=["object"]).columns

# Use ColumnTransformer to apply OneHotEncoding to categorical columns and leave numeric columns as they are
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_cols),
        (
            "cat",
            OneHotEncoder(handle_unknown="ignore"),
            categorical_cols,
        ),  # Ignore unknown categories
    ]
)

# Apply the preprocessing pipeline to the training data
X_train_processed = preprocessor.fit_transform(X_train)

# Apply the preprocessing pipeline to the test data
X_test_processed = preprocessor.transform(X_test)

# Define parameter grid for Logistic Regression with L2 regularization (ridge)
log_param_grid = {
    "C": [
        0.01,
        0.1,
        1,
        10,
        100,
    ],  # Regularization strength (inverse of regularization term)
    "solver": ["lbfgs", "liblinear"],  # Solvers that support regularization
}

# Apply GridSearchCV to find the best parameters for Logistic Regression
log_grid_search = GridSearchCV(
    estimator=LogisticRegression(),
    param_grid=log_param_grid,
    scoring="roc_auc",
    cv=5,
    n_jobs=-1,
)
log_grid_search.fit(X_train_processed, y_train)

# Get best Logistic Regression model
best_log_model = log_grid_search.best_estimator_

# Perform cross-validation on the best Logistic Regression model
log_cv_scores = cross_val_score(
    best_log_model, X_train_processed, y_train, cv=5, scoring="roc_auc"
)

# Predict and evaluate using the tuned model
log_y_pred = best_log_model.predict(X_test_processed)
log_y_pred_prob = best_log_model.predict_proba(X_test_processed)[:, 1]

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, log_y_pred)
precision = precision_score(y_test, log_y_pred)
recall = recall_score(y_test, log_y_pred)
f1 = f1_score(y_test, log_y_pred)
roc_auc = roc_auc_score(y_test, log_y_pred_prob)

# Calculate PR AUC
precision_vals, recall_vals, _ = precision_recall_curve(y_test, log_y_pred_prob)
pr_auc = auc(recall_vals, precision_vals)

# Create a DataFrame for metrics
metrics_df = pd.DataFrame(
    {
        "Metric": ["Accuracy", "Precision", "Recall", "F1 Score", "ROC AUC", "PR AUC"],
        "Value": [accuracy, precision, recall, f1, roc_auc, pr_auc],
    }
)

# Display the best model and metrics
print("Best Model Parameters:", log_grid_search.best_params_)
metrics_df

In [ ]:
# Plot the confusion matrix
conf_matrix = confusion_matrix(y_test, log_y_pred)
conf_matrix_reversed = np.flip(conf_matrix, axis=(0, 1))

plt.figure(figsize=(8, 6))
sns.heatmap(
    conf_matrix_reversed,
    annot=True,
    fmt="d",
    cmap="Blues",
    cbar=False,
    xticklabels=["Predicted Positive", "Predicted Negative"],
    yticklabels=["Actual Positive", "Actual Negative"],
)
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
# Plot Precision-Recall (PR) curve
plt.figure()
plt.plot(
    recall_vals,
    precision_vals,
    color="orange",
    lw=2,
    label=f"PR curve (AUC = {pr_auc:.2f})",
)
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall (PR) Curve")
plt.legend(loc="lower left")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.grid(True)
plt.show()

#### Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Define parameter grid for Decision Tree
tree_param_grid = {
    "max_depth": [None, 10, 20, 30, 40, 50],
    "min_samples_split": [2, 10, 20],
    "min_samples_leaf": [1, 5, 10],
    "criterion": ["gini", "entropy"],
}

# Apply GridSearchCV to find the best parameters for Decision Tree
tree_grid_search = GridSearchCV(
    estimator=DecisionTreeClassifier(),
    param_grid=tree_param_grid,
    scoring="roc_auc",
    cv=5,
    n_jobs=-1,
)
tree_grid_search.fit(X_train_processed, y_train)

# Get the best Decision Tree model
best_tree_model = tree_grid_search.best_estimator_

# Perform cross-validation on the best Decision Tree model
tree_cv_scores = cross_val_score(
    best_tree_model, X_train_processed, y_train, cv=5, scoring="roc_auc"
)

# Predict and evaluate using the tuned Decision Tree model
tree_y_pred = best_tree_model.predict(X_test_processed)
tree_y_pred_prob = best_tree_model.predict_proba(X_test_processed)[:, 1]

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, tree_y_pred)
precision = precision_score(y_test, tree_y_pred)
recall = recall_score(y_test, tree_y_pred)
f1 = f1_score(y_test, tree_y_pred)
roc_auc = roc_auc_score(y_test, tree_y_pred_prob)

# Calculate PR AUC
precision_vals, recall_vals, _ = precision_recall_curve(y_test, tree_y_pred_prob)
pr_auc = auc(recall_vals, precision_vals)

# Create a DataFrame for metrics
metrics_df = pd.DataFrame(
    {
        "Metric": ["Accuracy", "Precision", "Recall", "F1 Score", "ROC AUC", "PR AUC"],
        "Value": [accuracy, precision, recall, f1, roc_auc, pr_auc],
    }
)

# Display the best model and metrics
print("Best Model Parameters:", tree_grid_search.best_params_)
metrics_df

In [ ]:
# Plot the confusion matrix
conf_matrix = confusion_matrix(y_test, tree_y_pred)
conf_matrix_reversed = np.flip(conf_matrix, axis=(0, 1))

plt.figure(figsize=(8, 6))
sns.heatmap(
    conf_matrix_reversed,
    annot=True,
    fmt="d",
    cmap="Blues",
    cbar=False,
    xticklabels=["Predicted Positive", "Predicted Negative"],
    yticklabels=["Actual Positive", "Actual Negative"],
)
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
# Plot Precision-Recall (PR) curve
plt.figure()
plt.plot(
    recall_vals,
    precision_vals,
    color="orange",
    lw=2,
    label=f"PR curve (AUC = {pr_auc:.2f})",
)
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall (PR) Curve")
plt.legend(loc="lower left")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.grid(True)
plt.show()

#### Gradient Boost

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV, cross_validate
from sklearn.metrics import (
    make_scorer,
    accuracy_score,
    precision_score,
    recall_score,
    roc_auc_score,
    confusion_matrix,
)
import numpy as np

# Define the parameter grid for GridSearchCV
param_grid = {
    "n_estimators": [50, 100, 150],
    "learning_rate": [0.05, 0.1, 0.2],
    "max_depth": [3, 5, 7],
}

# Initialize the Gradient Boosting model
model = GradientBoostingClassifier(random_state=42)

# Define scoring metrics for cross-validation
scoring = {
    "accuracy": make_scorer(accuracy_score),
    "precision": make_scorer(precision_score),
    "recall": make_scorer(recall_score),
    "roc_auc": make_scorer(roc_auc_score, needs_proba=True),
}

# Initialize GridSearchCV with cross-validation
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring="recall",
    cv=5,
    n_jobs=-1,
    verbose=1,
)
grid_search.fit(X_train_scaled, y_train)

# Get the best model from grid search
best_model = grid_search.best_estimator_
print(f"Best Parameters: {grid_search.best_params_}")

# Perform cross-validation on the best model
cv_results = cross_validate(
    best_model, X_train_scaled, y_train, cv=5, scoring=scoring, return_train_score=False
)

# Calculate and print mean cross-validated metrics
accuracy_cv = np.mean(cv_results["test_accuracy"])
precision_cv = np.mean(cv_results["test_precision"])
recall_cv = np.mean(cv_results["test_recall"])
roc_auc_cv = np.mean(cv_results["test_roc_auc"])

print(f"\nCross-Validated Metrics for Best Model:")
print(f"Accuracy: {accuracy_cv:.4f}")
print(f"Precision: {precision_cv:.4f}")
print(f"Recall: {recall_cv:.4f}")
print(f"ROC AUC: {roc_auc_cv:.4f}")

# Fit the best model on the full training data and evaluate on test set
best_model.fit(X_train_scaled, y_train)
y_pred = best_model.predict(X_test_scaled)
y_pred_prob = best_model.predict_proba(X_test_scaled)[:, 1]

# Calculate metrics on test set
accuracy_test = accuracy_score(y_test, y_pred)
precision_test = precision_score(y_test, y_pred)
recall_test = recall_score(y_test, y_pred)
roc_auc_test = roc_auc_score(y_test, y_pred_prob)

# Confusion matrix for test set
conf_matrix = confusion_matrix(y_test, y_pred)
conf_matrix_reversed = np.flip(conf_matrix, axis=(0, 1))

In [ ]:
# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
conf_matrix_reversed = np.flip(conf_matrix, axis=(0, 1))
roc_auc = roc_auc_score(y_test, y_pred_prob)

# Calculate PR AUC
precision_vals, recall_vals, _ = precision_recall_curve(y_test, y_pred_prob)
pr_auc = auc(recall_vals, precision_vals)

# Create a DataFrame for metrics
metrics_df = pd.DataFrame(
    {
        "Metric": ["Accuracy", "Precision", "Recall", "F1 Score", "ROC AUC", "PR AUC"],
        "Value": [accuracy, precision, recall, f1, roc_auc, pr_auc],
    }
)

metrics_df

In [ ]:
# Plot the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(
    conf_matrix_reversed,
    annot=True,
    fmt="d",
    cmap="Blues",
    cbar=False,
    xticklabels=["Predicted Positive", "Predicted Negative"],
    yticklabels=["Actual Positive", "Actual Negative"],
)
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
# Plot Precision-Recall (PR) curve
plt.figure()
plt.plot(
    recall_vals,
    precision_vals,
    color="orange",
    lw=2,
    label=f"PR curve (AUC = {pr_auc:.2f})",
)
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall (PR) Curve")
plt.legend(loc="lower left")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.grid(True)
plt.show()

#### XGBoost

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV, cross_validate
from sklearn.metrics import (
    make_scorer,
    accuracy_score,
    precision_score,
    recall_score,
    roc_auc_score,
    confusion_matrix,
)
import numpy as np
import matplotlib.pyplot as plt

# Define the parameter grid with regularization hyperparameters
param_grid = {
    "n_estimators": [50, 100, 150],
    "learning_rate": [0.05, 0.1, 0.2],
    "max_depth": [3, 5, 7],
    "subsample": [0.8, 1.0],  # Regularization by subsampling
    "min_samples_split": [2, 5, 10],  # Minimum samples required to split a node
    "min_samples_leaf": [1, 2, 4],  # Minimum samples required at each leaf node
    "max_features": [
        "sqrt",
        "log2",
        None,
    ],  # Maximum number of features considered for each split
}

# Initialize the Gradient Boosting model
model = GradientBoostingClassifier(random_state=42)

# Define scoring metrics for cross-validation
scoring = {
    "accuracy": make_scorer(accuracy_score),
    "precision": make_scorer(precision_score),
    "recall": make_scorer(recall_score),
    "roc_auc": make_scorer(roc_auc_score, needs_proba=True),
}

# Initialize GridSearchCV with cross-validation
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring="recall",
    cv=5,
    n_jobs=-1,
    verbose=1,
)
grid_search.fit(X_train, y_train)

# Get the best model from grid search
best_model = grid_search.best_estimator_
print(f"Best Parameters: {grid_search.best_params_}")

# Perform cross-validation on the best model
cv_results = cross_validate(
    best_model, X_train, y_train, cv=5, scoring=scoring, return_train_score=False
)

# Calculate and print mean cross-validated metrics
accuracy_cv = np.mean(cv_results["test_accuracy"])
precision_cv = np.mean(cv_results["test_precision"])
recall_cv = np.mean(cv_results["test_recall"])
roc_auc_cv = np.mean(cv_results["test_roc_auc"])

print(f"\nCross-Validated Metrics for Best Model:")
print(f"Accuracy: {accuracy_cv:.4f}")
print(f"Precision: {precision_cv:.4f}")
print(f"Recall: {recall_cv:.4f}")
print(f"ROC AUC: {roc_auc_cv:.4f}")

# Fit the best model on the full training data and evaluate on test set
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)
y_pred_prob = best_model.predict_proba(X_test)[:, 1]

# Calculate metrics on test set
accuracy_test = accuracy_score(y_test, y_pred)
precision_test = precision_score(y_test, y_pred)
recall_test = recall_score(y_test, y_pred)
roc_auc_test = roc_auc_score(y_test, y_pred_prob)

# Confusion matrix for test set
conf_matrix = confusion_matrix(y_test, y_pred)
conf_matrix_reversed = np.flip(conf_matrix, axis=(0, 1))

In [ ]:
# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
conf_matrix_reversed = np.flip(conf_matrix, axis=(0, 1))
roc_auc = roc_auc_score(y_test, y_pred_prob)

# Calculate PR AUC
precision_vals, recall_vals, _ = precision_recall_curve(y_test, y_pred_prob)
pr_auc = auc(recall_vals, precision_vals)

# Create a DataFrame for metrics
metrics_df = pd.DataFrame(
    {
        "Metric": ["Accuracy", "Precision", "Recall", "F1 Score", "ROC AUC", "PR AUC"],
        "Value": [accuracy, precision, recall, f1, roc_auc, pr_auc],
    }
)

metrics_df

In [ ]:
# Plot the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(
    conf_matrix_reversed,
    annot=True,
    fmt="d",
    cmap="Blues",
    cbar=False,
    xticklabels=["Predicted Positive", "Predicted Negative"],
    yticklabels=["Actual Positive", "Actual Negative"],
)
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
# Plot Precision-Recall (PR) curve
plt.figure()
plt.plot(
    recall_vals,
    precision_vals,
    color="orange",
    lw=2,
    label=f"PR curve (AUC = {pr_auc:.2f})",
)
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall (PR) Curve")
plt.legend(loc="lower left")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.grid(True)
plt.show()

## Model Interpretation

SHAP is a method of local explanation that tells us how much each feature contributes to the model.

The larger the bar, the larger the contribution for the feature

In [ ]:
import shap
import matplotlib.pyplot as plt
import pandas as pd

# Ensure you use TreeExplainer for GradientBoostingClassifier
explainer = shap.TreeExplainer(best_model)

# Compute SHAP values
shap_values = explainer.shap_values(X_train)

# Convert SHAP values to DataFrame for analysis
shap_df = pd.DataFrame(shap_values, columns=X_train.columns)

# Identify the top 5 features by mean absolute SHAP value
top_features = shap_df.abs().mean().nlargest(5).index

# Filter data for the top features
X_train_top = X_train[top_features]

# Summary plot (bar chart) for top features
shap.summary_plot(shap_values, X_train, plot_type="bar", max_display=5, show=False)
plt.title("Top 5 Features - SHAP Feature Importance")
plt.tight_layout()
plt.show()

# Beeswarm plot for the top 5 features
shap.summary_plot(shap_values[:, : len(top_features)], X_train_top, show=False)
plt.title("Top 5 Features - SHAP Impact on Predictions")
plt.tight_layout()
plt.show()

## Export CSV

Selected Model:

In [ ]:
# export predicted probabilities
y_pred_prob = best_model.predict_proba(X_test)[:, 1]
y_pred_prob = pd.DataFrame(y_pred_prob, columns=["predicted_probability"])
y_pred_prob.to_csv("predicted_probabilities.csv", index=False)

## Export Best Model

In [ ]:
import pickle

# Save the model
with open("best_model.pkl", "wb") as file:
    pickle.dump(best_model, file)
print("Model saved to best_model.pkl")